<a href="https://colab.research.google.com/github/mehraj-alom/Deep_learrning/blob/main/tf_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mehrajalomtapadar_cats_in_cats_path = kagglehub.dataset_download('mehrajalomtapadar/cats-in-cats')
mehrajalomtapadar_dogs_in_dogs_path = kagglehub.dataset_download('mehrajalomtapadar/dogs-in-dogs')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

In [ ]:
for data in tf_dataset.take(4):
    print(data.numpy())

In [ ]:
tf_dataset = tf_dataset.filter(lambda x : x>0).map(lambda y : y/255).shuffle(3).batch(2)
for data in tf_dataset.take(4).as_numpy_iterator():
    print(data)

In [ ]:
import pandas as pd
df_cats = pd.read_csv("/kaggle/input/cats-in-cats/Cats.csv")
df_dogs = pd.read_csv("/kaggle/input/dogs-in-dogs/dog_images.csv")


In [ ]:
print("Cat CSV Columns:", df_cats.columns)
print("Dog CSV Columns:", df_dogs.columns)


In [ ]:

df_cats = df_cats.rename(columns={
    "Image" : "url"
})
df_cats["label"] = 0
df_dogs["label"] = 1

df_all = pd.concat([df_dogs[["url","label"]],df_cats[["url","label"]]],ignore_index=True)
file_path = df_all["url"].values
label = df_all["label"].values

dataset = tf.data.Dataset.from_tensor_slices((file_path,label))

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, label

pipeline = (
    dataset
    .shuffle(buffer_size=100)
    .map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)


In [ ]:
for file_paths , label in dataset.take(3).as_numpy_iterator():
    print(file_paths)
    print("***",label)